# Imports

In [ ]:
# Numerai API
from numerapi import NumerAPI

# data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# stats
from scipy.stats import spearmanr
from sklearn.metrics import r2_score, mean_squared_error

# machine learning models
from sklearn.base import BaseEstimator
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# other
import gc
import json
from tqdm import trange
from itertools import product
import functools
import random
from timeit import default_timer
import re
import time
from pprint import pprint
from copy import deepcopy
from varname import nameof
from datetime import datetime

# save variables
import pickle
import joblib

# my utils
from utils import rank_pct, numerai_score, exposure

# Download data

In [ ]:
napi = NumerAPI()
round = napi.get_current_round()

# filenames = napi.list_datasets()

napi.download_dataset('v4/live.parquet', f'v4/live_{round}.parquet')
napi.download_dataset('v4/live_int8.parquet', f'v4/live_int8_{round}.parquet')

# Features

- feature sets: `all`, `small`, `medium`, `v2_equivalent_features`, `v3_equivalent_features`, `fncv3_features`
- feature groups: `features_all[0:210]`, `features_all[210:420]`, `features_all[420:630]`, `features_all[630:840]`, `features_all[840:1050]`, `features_all[1050:1191]`

In [ ]:
with open('v4/FEATURES.json', 'r') as f:
    FEATURE_METADATA = json.load(f)
del f

FEATURES_L = list(FEATURE_METADATA['feature_stats'].keys())
FEATURES_M = FEATURE_METADATA['feature_sets']['medium']
FEATURES_S = FEATURE_METADATA['feature_sets']['small']
FEATURES_2 = FEATURE_METADATA['feature_sets']['v2_equivalent_features']
FEATURES_3 = FEATURE_METADATA['feature_sets']['v3_equivalent_features']
FEATURES_N = FEATURE_METADATA['feature_sets']['fncv3_features']

ERA = 'era'
DATA = 'data_type'
Y_TRUE = 'target_nomi_v4_20'
Y_PRED = 'target_prediction'
Y_RANK = 'prediction' 

X_COLS = FEATURES_L
COLUMNS = [ERA, DATA] + X_COLS + [Y_TRUE]

df_feature_metadata = pd.DataFrame(FEATURE_METADATA['feature_stats'])
df_feature_metadata

# Dataframes

## Train

In [ ]:
# df_trn = pd.read_parquet('v4/train_int8.parquet', columns=COLUMNS)
# df_trn[ERA] = df_trn[ERA].astype('int32')
# df_trn.info(memory_usage='deep')
# df_trn

## Validation + Test

In [ ]:
# df_val = pd.read_parquet('v4/validation_int8.parquet', columns=COLUMNS)
# df_val[ERA] = df_val[ERA].astype('int32')
# df_val.info(memory_usage='deep')
# df_val

## Live

In [ ]:
# df_liv = pd.read_parquet('v4/live_int8_{round}.parquet', columns=COLUMNS)
# df_liv.info(memory_usage='deep')
# df_liv

# Analyse data

## Number of examples as a function of the era

In [ ]:
# x_trn = df.groupby(ERA).size().index.values
# y_trn = df_trn.groupby(ERA).size().values
# x_val = df_val[df_val[DATA]=='validation'].groupby(ERA).size().index.values
# y_val = df_val[df_val[DATA]=='validation'].groupby(ERA).size().values
# x_tst = df_val[df_val[DATA]=='test'].groupby(ERA).size().index.values
# y_tst = df_val[df_val[DATA]=='test'].groupby(ERA).size().values

# fig, ax = plt.subplots()
# ax.plot(x_trn, y_trn, label='train')
# ax.plot(x_val, y_val, label='validation')
# ax.plot(x_tst, y_tst, label='test')
# ax.set_xlabel(ERA)
# ax.set_ylabel('number of examples')
# ax.legend()

In [ ]:
# del df_val
# del df_liv
# gc.collect()

## Feature correlation heatmap

In [ ]:
# feature_correlations = df_trn[df_trn[ERA]==1][FEATURES].corr()
# plt.figure(figsize = (8,8))
# plt.imshow(feature_correlations)
# for a in [210, 420, 630, 840, 1050]:
#     plt.axvline(a, color='orange')
#     plt.axhline(a, color='orange')

## Correlation of feature with target as a function of the era

In [ ]:
# def corrs_with_target(era):
#     return np.corrcoef(df_trn[df_trn[ERA]==era][[TARGET] + FEATURES].T)[0, 1:]

# eras = df_trn[ERA].unique()
# t_corrs = np.array([corrs_with_target(era) for era in eras])
# t_corrs = pd.DataFrame(t_corrs)
# t_corrs.rename(columns = dict(enumerate(FEATURES)), inplace=True)
# t_corrs.insert(0, ERA, eras)
# joblib.dump(t_corrs, 'saved-variables/t_corrs.pkl')
# t_corrs = joblib.load('saved-variables/t_corrs.pkl')

In [ ]:
# x = t_corrs[ERA]
# y = t_corrs['feature_untidy_withdrawn_bargeman']

# fig, ax = plt.subplots()
# ax.plot(x, y)
# ax.set_xlabel(ERA)
# ax.set_ylabel('correlation with target')

# Notes for testing models

Performance metrics:

- correlation
- rank-correlation / spearman-correlation
- `sklearn.metrics.r2_score`
- `sklearn.metrics.mean_squared_error`

Models worth trying at first

- `sklearn.linear_model.LinearRegression()`
- `sklearn.linear_model.LogisticRegression()` (This doesn't work, it's only for classification - but some example uses it?)
- `sklearn.linear_model.SGDRegressor()` (Stochastic Gradient Descent regressor)
- `sklearn.linear_model.Lasso()`
- `sklearn.linear_model.ElasticNet()`
- `sklearn.linear_model.Ridge()`
- `sklearn.svm.SVR(kernel='rbf')` (Support Vector Machine / Regression)
- `sklearn.svm.SVR(kernel='linear')`
- `lightgbm.LGBMRegressor()`
- `xgboost.XGBRegressor()`

Ensembles

- `sklearn.ensemble.RandomForestRegressor()`
- `sklearn.ensemble.ExtraTreesRegressor()`
- `sklearn.ensemble.BaggingRegressor()`
- `sklearn.ensemble.AdaBoostRegressor()`
- `sklearn.ensemble.GradientBoostingRegressor()`

# EraSubsampler class

In [ ]:
class EraSubsampler(BaseEstimator):
    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, X, y, eras):
        X, y = check_X_y(X, y, accept_sparse=True)
        e0 = eras.min()
        e1 = eras.max() + 1
        self.model = [deepcopy(self.estimator) for i in range(4)]
        for i in trange(4):
            self.model[i].fit(X[eras.isin(np.arange(e0 + i, e1, 4))], 
                              y[eras.isin(np.arange(e0 + i, e1, 4))])
        self.is_fitted_ = True
        return self

    def predict(self, X):
        X = check_array(X, accept_sparse=True)
        check_is_fitted(self, 'is_fitted_')
        y_pred = 0
        for i in trange(4):
            y_pred += self.model[i].predict(X)
        y_pred /= 4
        return y_pred

    # TODO: make score function be the numerai_score (depends on groups)
    def score(self, X, y):
        return r2_score(y, self.predict(X))

# XGBoost

In [ ]:
# training set
X_COLS = FEATURES_S
COLUMNS = [ERA, DATA] + X_COLS + [Y_TRUE]
df_trn = pd.read_parquet('v4/train_int8.parquet', columns=COLUMNS)
df_trn[ERA] = df_trn[ERA].astype('int32')
df_trn = df_trn[df_trn[ERA] <= 20]

In [ ]:
# model
params = {
    'n_estimators': 2000,
    'learning_rate': 0.01,
    'max_depth': 5,
    'max_leaves': 2**5,
    'colsample_bytree': 0.1,
    'gpu_id': 0,
    'tree_method': 'gpu_hist',
}
xgb = EraSubsampler(XGBRegressor(**params))

In [ ]:
# train
xgb.fit(df_trn[X_COLS], df_trn[Y_TRUE], df_trn[ERA])

In [ ]:
# predict and score on training set
df_trn[Y_PRED] = xgb.predict(df_trn[X_COLS])
df_trn[Y_RANK] = df_trn[Y_PRED].rank(pct=True)
ns_trn = numerai_score(df_trn[Y_TRUE], df_trn[Y_PRED], df_trn[ERA])

In [ ]:
# define validation set
df_val = pd.read_parquet('v4/validation_int8.parquet', columns=COLUMNS)
df_val[ERA] = df_val[ERA].astype('int32')
# df_val = df_val[df_val[ERA] <= 575 + 20]

# predict and score on validation set
df_val[Y_PRED] = xgb.predict(df_val[X_COLS])
df_val[Y_RANK] = df_val[Y_PRED].rank(pct=True)
ns_val = numerai_score(df_val[Y_TRUE], df_val[Y_PRED], df_val[ERA])

In [ ]:
# define live set
df_liv = pd.read_parquet(f'v4/live_int8_{round}.parquet', columns=COLUMNS)

# predict on validation set
df_liv[Y_PRED] = xgb.predict(df_liv[X_COLS])
df_liv[Y_RANK] = df_liv[Y_PRED].rank(pct=True)

In [ ]:
# save variables
now = datetime.now().strftime('%Y%m%d%H%M')
joblib.dump(xgb, f'saved-variables/xgb_{now}.pkl')
df_val[Y_RANK].to_csv(f'predictions/validation_predictions_{round}_{now}.csv')
df_liv[Y_RANK].to_csv(f'predictions/live_predictions_{round}_{now}.csv')

# Grid Search test

In [ ]:
# param_grid = {'estimator__' + k: [v] for k, v in params.items()}
# xgbGS = GridSearchCV(EraSubsampler(XGBRegressor()), param_grid)

# xgbGS.fit(df_trn[X_COLS], df_trn[Y_TRUE], groups=df_trn[ERA], **{'eras': df_trn[ERA]})

# bst = xgbGS.best_estimator_

# df_trn['bst_' + Y_PRED] = bst.predict(df_trn[X_COLS])
# df_trn['bst_' + Y_RANK] = df_trn['bst_' + Y_PRED].rank(pct=True)
# ns_bst = numerai_score(df_trn[Y_TRUE], df_trn['bst_' + Y_PRED], df_trn[ERA])